In [1]:
path = "../../../MSKCC/Chodera Lab/feedstock/FreeSolv-0.51/database.txt"
from pickle import load
with open(path, 'r') as f:
    freesolv = f.read()
from tqdm import tqdm

In [2]:
legend = freesolv.split('\n')[2].split('; ')
print(list(zip(range(len(legend)), legend)))

[(0, '# compound id (and file prefix)'), (1, 'SMILES'), (2, 'iupac name (or alternative if IUPAC is unavailable or not parseable by OEChem)'), (3, 'experimental value (kcal/mol)'), (4, 'experimental uncertainty (kcal/mol)'), (5, 'Mobley group calculated value (GAFF) (kcal/mol)'), (6, 'calculated uncertainty (kcal/mol)'), (7, 'experimental reference (original or paper this value was taken from)'), (8, 'calculated reference'), (9, 'text notes.')]


In [3]:
db = []
for entry in freesolv.split('\n')[3:-1]:
    db.append(entry.split('; '))

In [4]:
smiles_list = [entry[1] for entry in db]

In [5]:
sorted(smiles_list, key=len)

['S',
 'N',
 'C',
 'CO',
 'CC',
 'CI',
 'CS',
 'CN',
 'NN',
 'CF',
 'CCI',
 'CCS',
 'CCC',
 'C=O',
 'CCO',
 'CCN',
 'CCl',
 'CNC',
 'C=C',
 'COC',
 'CBr',
 'CSC',
 'CCCI',
 'CCCS',
 'CCCC',
 'CC=O',
 'CCSC',
 'CCCl',
 'CC=C',
 'CCCO',
 'CCOC',
 'CC#N',
 'CSSC',
 'CC#C',
 'CCCN',
 'CCBr',
 'CCCCO',
 'CCOCC',
 'CCCCS',
 'CCC=C',
 'COC=O',
 'CCCCl',
 'CCCOC',
 'C1CC1',
 'COCOC',
 'CCC#N',
 'CCSCC',
 'CCCCC',
 'COCCO',
 'C=CCl',
 'CCC=O',
 'C(I)I',
 'COCCN',
 'CCCBr',
 'C=CCO',
 'CCNCC',
 'CCCCI',
 'CCC#C',
 'CCCCN',
 'CC(C)I',
 'CCOC=O',
 'CCCCBr',
 'CC(F)F',
 'C(F)Cl',
 'CCCCCl',
 'CCCC=C',
 'CCCC#C',
 'COCCOC',
 'C(CO)O',
 'C=CC=C',
 'CC(C)C',
 'CCCCCO',
 'CCSSCC',
 'C1CNC1',
 'CCCCCC',
 'C=CCCl',
 'CCCCCN',
 'CC(C)O',
 'CCCC#N',
 'CCCCCI',
 'CCCC=O',
 'CN(C)C',
 'CCOCCO',
 'CC(C)Cl',
 'CCCCC#C',
 'CCC(C)C',
 'CCCCCCl',
 'CC(C)CO',
 'CCCNCCC',
 'c1ccsc1',
 'CC(=C)C',
 'CC(=O)O',
 'CC(C)OC',
 'CCCCCCO',
 'CCCCCBr',
 'C(Cl)Cl',
 'CC(=O)C',
 'CCCOCCO',
 'CCCCC#N',
 'CCCSCCC',
 'CCCCC=C',
 

In [6]:
import openeye

In [7]:
from openeye import oeomega  # Omega toolkit
omega = oeomega.OEOmega()
omega.SetMaxConfs(800)  # Best-practice? Expensive...
omega.SetIncludeInput(False)
omega.SetStrictStereo(True)  # Refuse to generate conformers if stereochemistry not provided

from openforcefield.typing.engines.smirnoff import ForceField, generateTopologyFromOEMol
from openeye import oechem  # OpenEye Python toolkits

from openeye import oequacpac  # Charge toolkit

ff = ForceField('forcefield/smirnoff99Frosst.offxml')

def generate_oemol(smiles):
    """Add hydrogens, assign charges, generate conformers, and return molecule"""
    mol = oechem.OEMol()
    chargeEngine = oequacpac.OEAM1BCCCharges()
    oechem.OEParseSmiles(mol, smiles)
    oechem.OEAddExplicitHydrogens(mol)
    status = omega(mol)
    if not status: print("Something went wrong in `generate_oemol({})!".format(smiles))
    oechem.OETriposAtomNames(mol)
    oequacpac.OESetNeutralpHModel(mol)
    oequacpac.OEAssignCharges(mol, chargeEngine)
    _ = generateTopologyFromOEMol(mol)
    return mol


def generate_mol_top_sys_pos(smiles):
    """Generate an openmm topology, openmm system, and coordinate array from a smiles string"""
    print(smiles)
    mol = generate_oemol(smiles)

    coord_dict = mol.GetCoords()
    positions = np.array([coord_dict[key] for key in coord_dict])

    topology = generateTopologyFromOEMol(mol)

    system = ff.createSystem(topology, [mol])

    return mol, topology, system, positions

In [8]:
sorted_smiles = sorted(list(set(smiles_list)), key=len)[::-1]

In [9]:
from tqdm import tqdm
import numpy as np
mol_top_sys_pos_list = []

for smiles in tqdm(sorted_smiles):
    mol_top_sys_pos_list.append(tuple(generate_mol_top_sys_pos(smiles)))

  0%|          | 0/642 [00:00<?, ?it/s]

C1[C@@H]2[C@H]3[C@@H]([C@H]1[C@H]4[C@@H]2O4)[C@@]5(C(=C([C@]3(C5(Cl)Cl)Cl)Cl)Cl)Cl


  0%|          | 1/642 [00:01<15:26,  1.45s/it]

C1[C@H]([C@@H]2[C@H]([C@H]1Cl)[C@]3(C(=C([C@@]2(C3(Cl)Cl)Cl)Cl)Cl)Cl)Cl


  0%|          | 2/642 [00:02<11:48,  1.11s/it]

...

COC
CS
CF
NN
CN


 99%|█████████▉| 638/642 [3:15:30<01:13, 18.39s/it]

CC
CO
CI
N
S


100%|██████████| 642/642 [3:15:31<00:00, 18.27s/it]

C


In [10]:
from pickle import dump
with open('mol_top_sys_pos.pkl', 'wb') as f:
    dump(mol_top_sys_pos_list, f)

In [12]:
total_n_atoms = 0
for (mol, top, sys, pos) in mol_top_sys_pos_list:
    total_n_atoms += len(pos)
print(total_n_atoms)

11621


In [14]:
from pickle import dump
with open('sorted_smiles.pkl', 'wb') as f:
    dump(sorted_smiles, f)